In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install keras-tuner --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.2 MB/s eta 0:00:00


In [23]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
import keras_tuner
import keras
from tensorflow.keras import layers,Sequential
from keras_tuner import RandomSearch

In [6]:
df = pd.read_csv('/content/drive/MyDrive/deep learning/datasets/ANN datasets/diabetes_risk_prediction_dataset.csv')
df

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
3,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
4,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,39,Female,Yes,Yes,Yes,No,Yes,No,No,Yes,No,Yes,Yes,No,No,No,Positive
516,48,Female,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,Yes,Yes,No,No,No,Positive
517,58,Female,Yes,Yes,Yes,Yes,Yes,No,Yes,No,No,No,Yes,Yes,No,Yes,Positive
518,32,Female,No,No,No,Yes,No,No,Yes,Yes,No,Yes,No,No,Yes,No,Negative


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Age                 520 non-null    int64 
 1   Gender              520 non-null    object
 2   Polyuria            520 non-null    object
 3   Polydipsia          520 non-null    object
 4   sudden weight loss  520 non-null    object
 5   weakness            520 non-null    object
 6   Polyphagia          520 non-null    object
 7   Genital thrush      520 non-null    object
 8   visual blurring     520 non-null    object
 9   Itching             520 non-null    object
 10  Irritability        520 non-null    object
 11  delayed healing     520 non-null    object
 12  partial paresis     520 non-null    object
 13  muscle stiffness    520 non-null    object
 14  Alopecia            520 non-null    object
 15  Obesity             520 non-null    object
 16  class               520 no

In [8]:
df.describe()

,Age
count,520.000000
mean,48.028846
std,12.151466
min,16.000000
25%,39.000000
50%,47.500000
75%,57.000000
max,90.000000


In [9]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for column in X.columns:
  X[column] = le.fit_transform(X[column])
y = le.fit_transform(y)

In [11]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
X = pca.fit_transform(X)

In [12]:
X.shape

(520, 10)

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = 0.3,random_state = 1)

In [18]:
from keras.layers import Dense
def build_model(hp):
  model = Sequential()
  for i in range(hp.Int('num_layers',min_value = 2,max_value = 10)):
    if i==0 :
      model.add(Dense(units = hp.Int('units' + str(i),min_value = 20,max_value = 200),
                      kernel_initializer=hp.Choice('kernel'+str(i),values = ['HeUniform','HeNormal'])
                      ,activation = 'relu'))
      keras.layers.Dropout = hp.Choice('dropouts'+str(i),values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
    else :
            model.add(Dense(units = hp.Int('units' + str(i),min_value = 20,max_value = 200),
                      kernel_initializer=hp.Choice('kernel'+str(i),values = ['HeUniform','HeNormal'])
                      ,activation = 'relu'))
            keras.layers.Dropout = hp.Choice('dropouts'+str(i),values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer = hp.Choice('optimizers',values = ['rmsprop','adam']),loss = 'binary_crossentropy',metrics = ['accuracy'])

  return model

In [24]:
tuner = RandomSearch(
    build_model,

    objective = 'val_accuracy',
    directory = "aiml",
)

Reloading Tuner from aiml/untitled_project/tuner0.json


In [25]:
tuner.search(X_train,y_train,epochs = 15,validation_data = (X_val,y_val))

Trial 31 Complete [00h 00m 04s]
val_accuracy: 0.8461538553237915

Best val_accuracy So Far: 0.9166666865348816
Total elapsed time: 00h 02m 16s


In [26]:
tuner.get_best_hyperparameters()[0]

In [27]:
model = tuner.get_best_models()[0]

In [28]:
model.fit(X_train,y_train,epochs = 50,validation_data = (X_val,y_val))

Epoch 1/50
12/12 [==============================] - 1s 18ms/step - loss: 0.3484 - accuracy: 0.9093 - val_loss: 0.3434 - val_accuracy: 0.8782
Epoch 2/50
12/12 [==============================] - 0s 6ms/step - loss: 0.3370 - accuracy: 0.8984 - val_loss: 0.3107 - val_accuracy: 0.9103
Epoch 3/50
12/12 [==============================] - 0s 6ms/step - loss: 0.3243 - accuracy: 0.8956 - val_loss: 0.3038 - val_accuracy: 0.9038
Epoch 4/50
12/12 [==============================] - 0s 7ms/step - loss: 0.3039 - accuracy: 0.9176 - val_loss: 0.2957 - val_accuracy: 0.9038
Epoch 5/50
12/12 [==============================] - 0s 6ms/step - loss: 0.2919 - accuracy: 0.9341 - val_loss: 0.2848 - val_accuracy: 0.9295
Epoch 6/50
12/12 [==============================] - 0s 6ms/step - loss: 0.2851 - accuracy: 0.9148 - val_loss: 0.2707 - val_accuracy: 0.9231
Epoch 7/50
12/12 [==============================] - 0s 7ms/step - loss: 0.2702 - accuracy: 0.9286 - val_loss: 0.2632 - val_accuracy: 0.9167
Epoch 8/50
12/12 [=